In [2]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [4]:
%cd "ctorch211217/MiniCTorch_Prototype/notebook"

/content/drive/My Drive/Colab Notebooks/ctorch211217/MiniCTorch_Prototype/notebook


In [5]:
%matplotlib inline
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from tqdm import tqdm, trange

In [6]:
DEVICE = 'cpu' #torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
BATCH_SIZE = 4
TEST_BATCH_SIZE = 5

train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./fmnist', train=True, download=True,
    transform=transforms.ToTensor()), batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./fmnist', train=False, download=True,
    transform=transforms.ToTensor()), batch_size=TEST_BATCH_SIZE, shuffle=False)

TRAIN_SIZE  = len(train_loader.dataset)
TEST_SIZE   = len(test_loader.dataset)
NUM_BATCHES = len(train_loader)
NUM_TEST_BATCHES = len(test_loader)

CLASSES = 10
TRAIN_EPOCHS = 1
SAMPLES = 2
TEST_SAMPLES = 10


In [9]:
class Gaussian(object):
    def __init__(self, mu, rho):
        super().__init__()
        self.mu = mu
        self.rho = rho
        self.normal = torch.distributions.Normal(0,1)
    
    @property
    def sigma(self):
        return torch.log1p(torch.exp(self.rho))
    
    def sample(self):
        epsilon = self.normal.sample(self.rho.size()).to(DEVICE)
        return self.mu + self.sigma * epsilon
    
    def log_prob(self, input):
        return (- math.log(math.sqrt(2 * math.pi))
                - torch.log(self.sigma)
                - ((input - self.mu) ** 2) / (2 * self.sigma ** 2)).sum()

In [10]:
class ScaleMixtureGaussian(object):
    def __init__(self, pi, sigma1, sigma2):
        super().__init__()
        self.pi = pi
        self.sigma1 = sigma1
        self.sigma2 = sigma2
        self.gaussian1 = torch.distributions.Normal(0,sigma1)
        self.gaussian2 = torch.distributions.Normal(0,sigma2)
    
    def log_prob(self, input):
        prob1 = torch.exp(self.gaussian1.log_prob(input))
        prob2 = torch.exp(self.gaussian2.log_prob(input))
        return (torch.log(self.pi * prob1 + (1-self.pi) * prob2)).sum()

In [11]:
class BayesianLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        # Weight parameters
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-0.2, 0.2))
        self.weight_rho = nn.Parameter(torch.Tensor(out_features, in_features).uniform_(-5,-4))
        self.weight = Gaussian(self.weight_mu, self.weight_rho)
        # Bias parameters
        self.bias_mu = nn.Parameter(torch.Tensor(out_features).uniform_(-0.2, 0.2))
        self.bias_rho = nn.Parameter(torch.Tensor(out_features).uniform_(-5,-4))
        self.bias = Gaussian(self.bias_mu, self.bias_rho)
        # Prior distributions
        PI = 0.5
        SIGMA_1 = torch.FloatTensor([math.exp(-0)]).to(DEVICE)
        SIGMA_2 = torch.FloatTensor([math.exp(-6)]).to(DEVICE)
        self.weight_p = ScaleMixtureGaussian(PI, SIGMA_1, SIGMA_2)
        self.bias_p   = ScaleMixtureGaussian(PI, SIGMA_1, SIGMA_2)
        self.log_p = 0
        self.log_q = 0

    def forward(self, input, sample=False,):
        if self.training or sample:
            weight = self.weight.sample()
            bias = self.bias.sample()
        else:
            weight = self.weight.mu
            bias = self.bias.mu
        self.log_p = self.weight_p.log_prob(weight) + self.bias_p.log_prob(bias)
        self.log_q = self.weight.log_prob(weight) + self.bias.log_prob(bias)
        return F.linear(input, weight, bias)
    

In [12]:
SAMPLES=1
DEVICE='cpu'
class BBBLoss(nn.Module):
  def __init__(self, samples=SAMPLES):
    super().__init__()
    self.samples=samples

  def forward(self, x, target, model):
    #print("loss forward")
    #print("x",x)
    #print("target",target)
    outputs, log_ps, log_qs = model.sample(x,samples=self.samples)
    #print("outputs",outputs)
    #print("log_ps",log_ps)
    #print("log_qs",log_qs)
    log_p = log_ps.mean()
    log_q = log_qs.mean()
    #print("log_ps2",log_ps)
    #print("log_qs2",log_qs)
    nll = F.nll_loss(outputs.mean(0), target, size_average=False)
    loss = (log_q - log_p)/NUM_BATCHES + nll
    return loss
    
class BayesianNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = BayesianLinear(28*28, 400)
        self.l2 = BayesianLinear(400, 400)
        self.l3 = BayesianLinear(400, 10)
    
    def forward(self, x, sample=False):
        #print("net x0",x)
        x = x.view(-1, 28*28)
        #print("net x1",x)
        x = F.relu(self.l1(x, sample))
        #print("net x2",x)
        x = F.relu(self.l2(x, sample))
        #print("net x3",x)
        x = F.log_softmax(self.l3(x, sample), dim=1)
        #print("net x4",x)
        return x
    """ #220107 mod
    def sample(self, x, samples=SAMPLES):
      outputs = torch.zeros(samples, BATCH_SIZE, CLASSES).to(DEVICE)
      log_ps = torch.zeros(samples).to(DEVICE)
      log_qs = torch.zeros(samples).to(DEVICE)
      for i in range(samples):
          outputs[i] = self(x, sample=True)
          log_ps[i] = self.log_p()
          log_qs[i] = self.log_q()
      return outputs, log_ps, log_qs
    """
    def sample(self, x, samples=SAMPLES):
      outputs = [] #torch.zeros(samples, BATCH_SIZE, CLASSES).to(DEVICE)
      log_ps  = [] #torch.zeros(samples).to(DEVICE)
      log_qs  = [] #torch.zeros(samples).to(DEVICE)
      for i in range(samples):
          outputs.append(self(x, sample=True))
          log_ps.append(self.log_p())
          log_qs.append(self.log_q())
      outputs = torch.stack(outputs)
      log_ps  = torch.stack(log_ps)
      log_qs  = torch.stack(log_qs)
      return outputs, log_ps, log_qs
    
    def log_p(self):
        return self.l1.log_p \
               + self.l2.log_p \
               + self.l3.log_p
    
    def log_q(self):
        return self.l1.log_q \
               + self.l2.log_q \
               + self.l3.log_q
    

class Net(nn.Module):
  def __init__(self, target, samples=SAMPLES):
    super().__init__()
    self.samples=samples
    self.net = BayesianNetwork()
    self.net.train()
    self.loss_func = BBBLoss()
    self.target=target
    print("samples",samples)
    print("target",target)

  def forward(self, x):
    loss = self.loss_func(x, self.target, self.net)
    return loss


In [ ]:
#learning loop
model=Net(None)
model.to(DEVICE)
optimizer = optim.Adam(model.parameters())

for epoch in range(TRAIN_EPOCHS):
  sum_loss=0
  sum_count=0
  for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
    data, target = data.to(DEVICE), target.to(DEVICE)
    model.zero_grad()
    model.target=target
    loss = model(data)
    loss.backward()
    optimizer.step()
    sum_loss+=loss.item()
    sum_count+=1
  avg_loss=sum_loss/sum_count
  print(avg_loss)

In [13]:
!pip install lark-parser

     |████████████████████████████████| 103 kB 6.7 MB/s 


In [ ]:
#inmport minctorch
! pip install git+https://github.com/kojima-r/MiniCTorch_Prototype.git

In [14]:
project = 'bbb'
folder = './bbb'
json_path = folder + '/' + project +'.json'

In [15]:
! mkdir -p bbb

In [16]:
! jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

Traceback (most recent call last):
  File "/usr/local/bin/jupyter-notebook", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "</usr/local/lib/python2.7/dist-packages/decorator.pyc:decorator-gen-7>", line 2, in initialize
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 87, in catch_config_error
    return method(app, *args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/notebook/notebookapp.py", line 1368, in initialize
    self.init_webapp()
  File "/usr/local/lib/python2.7/dist-packages/notebook/notebookapp.py", line 1188, in init_webapp
    self.http_server.listen(port, self.ip)
  File "/usr/local/lib/python2.7/dist-packages/torna

In [17]:
def generate_json( json_path, x,y ):
  with torch.no_grad():
    model=Net(y)
    print("[SAVE]", json_path )
    minictorch.trace( model, x, json_path )
  return model

In [18]:
import sys
sys.path.append("../")
import minictorch

torch.manual_seed( 1 )

for data, target in train_loader:
  x=data
  y=target
  break

model = generate_json( json_path, x, y)

samples 1
target tensor([5, 4, 9, 4])
[SAVE] ./bbb/bbb.json


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.7/dist-packages/torch/jit/_trace.py:991: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Scalars are not close!

Absolute difference: 1.512237548828125 (up to 1e-05 allowed)
Relative difference: 0.010075795693127533 (up to 1e-05 allowed)
  _module_class,


skip: Net/BayesianNetwork[net]/BayesianLinear[l1]/weight_rho/l1
skip: Net/BayesianNetwork[net]/BayesianLinear[l1]/weight_rho/l1
skip: Net/BayesianNetwork[net]/BayesianLinear[l1]/weight_rho/l1
skip: Net/BayesianNetwork[net]/BayesianLinear[l1]/weight_rho/l1
skip: Net/BayesianNetwork[net]/BayesianLinear[l2]/weight_rho/l2
skip: Net/BayesianNetwork[net]/BayesianLinear[l2]/weight_rho/l2
skip: Net/BayesianNetwork[net]/BayesianLinear[l2]/weight_rho/l2
skip: Net/BayesianNetwork[net]/BayesianLinear[l2]/weight_rho/l2
skip: Net/BayesianNetwork[net]/BayesianLinear[l3]/weight_rho/l3
skip: Net/BayesianNetwork[net]/BayesianLinear[l3]/weight_rho/l3
skip: Net/BayesianNetwork[net]/BayesianLinear[l3]/weight_rho/l3
skip: Net/BayesianNetwork[net]/BayesianLinear[l3]/weight_rho/l3


In [19]:
x.shape

torch.Size([4, 1, 28, 28])

In [20]:
minictorch.convert_all(project, folder, model, json_path, x)

[JSON] ./bbb/bbb.json
{'name': 'Net/BayesianNetwork[net]/BayesianLinear[l1]/weight_mu/weight_mu.1', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [104, 26], 'sorted_id': 5}
param: l1_weight_mu  - str loop  5000  /  39199
param: l1_weight_mu  - str loop  10000  /  39199
param: l1_weight_mu  - str loop  15000  /  39199
param: l1_weight_mu  - str loop  20000  /  39199
param: l1_weight_mu  - str loop  25000  /  39199
param: l1_weight_mu  - str loop  30000  /  39199
param: l1_weight_mu  - str loop  35000  /  39199
{'name': 'Net/BayesianNetwork[net]/BayesianLinear[l1]/weight_rho/weight_rho.1', 'op': 'prim::GetAttr', 'in': [], 'output_id': 0, 'shape': [], 'out': [7, 13, 11, 106, 99], 'sorted_id': 6}
param: l1_weight_rho  - str loop  5000  /  39199
param: l1_weight_rho  - str loop  10000  /  39199
param: l1_weight_rho  - str loop  15000  /  39199
param: l1_weight_rho  - str loop  20000  /  39199
param: l1_weight_rho  - str loop  25000  /  39199
param: l1_weight_rho  - st

In [21]:
C_SRC = './bbb/bbb.cpp ./bbb/bbb_param.cpp'
#C_OPT = '-I ./MiniCTorch_Prototype/src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include'
C_OPT = '-I ../src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include'
C_OUT = './bbb/bbb'
!echo "SRC = "$C_SRC
!echo "OPT = "$C_OPT
!echo "OUT = "$C_OUT
!g++ -std=c++14 $C_SRC $C_OPT -lcblas -o $C_OUT

SRC = ./bbb/bbb.cpp ./bbb/bbb_param.cpp
OPT = -I ../src -I ./xtensor/include -I ./xtensor-blas/include -I ./xtl/include -I ./json/include
OUT = ./bbb/bbb


In [22]:
!./bbb/bbb

### forward computation ...
 161.495926
### backward computation ...
input_grad{{{{-8.654231e-04, -4.269364e-04, -4.211752e-03, ..., -2.047406e-03,  2.695079e-03, 
     3.625368e-03},
   { 8.013294e-03,  4.145207e-04, -9.661670e-03, ..., -2.831195e-03,  8.489743e-03, 
     5.248327e-03},
   { 3.957163e-03, -2.172270e-03,  2.272172e-03, ..., -1.064479e-02,  1.419934e-02, 
    -3.863031e-03},
   ..., 
   {-2.779534e-03, -8.126297e-03,  7.000541e-03, ..., -3.983426e-03, -4.735980e-03, 
    -1.767182e-03},
   {-4.776264e-03,  1.391060e-03, -5.851083e-03, ...,  3.595551e-03,  8.324711e-03, 
    -8.183821e-03},
   { 5.143474e-03,  1.090845e-02,  1.098170e-02, ..., -1.102129e-02, -2.885246e-03, 
    -3.514630e-04}}},
 {{{-7.371405e-06, -7.266106e-07,  3.269969e-05, ...,  4.496270e-05, -1.787129e-05, 
    -1.116919e-05},
   {-3.422331e-06, -2.316466e-05,  3.707759e-05, ..., -8.850149e-06,  3.690299e-05, 
    -1.295785e-06},
   {-2.464392e-06, -2.616125e-05, -2.027304e-05, ...,  1.444467e-05, -

In [23]:
# check code
model.train()
z = model( x )
print(z)
z.sum().backward()
print(x.grad)

tensor(146.8067, grad_fn=<AddBackward0>)
None


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [ ]:
xx = torch.tensor([-1 + 1j, -2 + 2j, 3 - 3j])
print(xx)
yy = xx.conj()
print(yy)
zz = yy.imag
print(zz)
print(zz.is_neg())

tensor([-1.+1.j, -2.+2.j,  3.-3.j])
tensor([-1.-1.j, -2.-2.j,  3.+3.j])
tensor([-1., -2.,  3.])
True


In [ ]:
out = yy.resolve_neg()
print(out)
print( out.is_neg())

tensor([-1.-1.j, -2.-2.j,  3.+3.j])
False
